In [1]:
%pylab inline
from classy import *

Populating the interactive namespace from numpy and matplotlib
Version:  0.0.26


In [2]:
from NumPyNet.network import Network
from NumPyNet.layers.connected_layer import Connected_layer
from NumPyNet.layers.cost_layer import Cost_layer
from NumPyNet.optimizer import Adam,SGD
from NumPyNet.utils import to_categorical
from NumPyNet.utils import from_categorical
from NumPyNet.metrics import mean_accuracy_score

In [3]:
data=load_excel('data/iris.xls')
data_train,data_test=split(data,test_size=0.2)

iris.data 151 5
150 vectors of length 4
Feature names: 'petal length in cm', 'petal width in cm', 'sepal length in cm', 'sepal width in cm'
Target values given.
Target names: 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'
Mean:  [3.75866667 1.19866667 5.84333333 3.054     ]
Median:  [4.35 1.3  5.8  3.  ]
Stddev:  [1.75852918 0.76061262 0.82530129 0.43214658]
Original vector shape:  (150, 4)
Train vector shape:  (120, 4)
Test vector shape:  (30, 4)


In [4]:
X=data.vectors
Y=data.targets

In [5]:
X.shape

(150, 4)

In [6]:
Y.shape

(150,)

In [7]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
one_hot_y=to_categorical(Y)
one_hot_y[40:60,:]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [9]:
def accuracy (y_true, y_pred):
    '''
    Temporary metrics to overcome "from_categorical" missing in standard metrics
    '''
    from NumPyNet.metrics import mean_accuracy_score
    truth = from_categorical(y_true)
    predicted = from_categorical(y_pred)
    return mean_accuracy_score(truth, predicted)


In [10]:
X_train=data_train.vectors
y_train=data_train.targets
one_hot_y_train=to_categorical(y_train)

X_test=data_test.vectors
y_test=data_test.targets
one_hot_y_test=to_categorical(y_test)

In [11]:
# Reshape the data according to a 4D tensor
num_samples, size = X_test.shape
num_samples, size

num_classes=one_hot_y_test.shape[1]

X_test = X_test.reshape(num_samples, 1, 1, size)
one_hot_y_test = one_hot_y_test.reshape(num_samples,1,1,-1)

X_test.shape,one_hot_y_test.shape

((30, 1, 1, 4), (30, 1, 1, 3))

In [12]:
# Reshape the data according to a 4D tensor
num_samples, size = X_train.shape
num_samples, size

num_classes=one_hot_y_train.shape[1]

X_train = X_train.reshape(num_samples, 1, 1, size)
one_hot_y_train = one_hot_y_train.reshape(num_samples,1,1,-1)

X_train.shape,one_hot_y_train.shape

((120, 1, 1, 4), (120, 1, 1, 3))

In [13]:
batch=len(y_train)
step = batch

In [14]:
model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Connected_layer(outputs=4, activation='relu'))
model.add(Connected_layer(outputs=num_classes, activation='linear'))
model.add(Cost_layer(cost_type='mse'))

#model.compile(optimizer=SGD(lr=0.1, decay=0.03, lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])

In [15]:
model.summary()

layer       filters  size              input                output
   0 input                   120 x   1 x   1 x   4   ->   120 x   1 x   1 x   4
   1 connected               120 x   1 x   1 x   4   ->   120 x   4
   2 connected               120 x   1 x   1 x   4   ->   120 x   3
   3 cost                    120 x   1 x   1 x   3   ->   120 x   1 x   1 x   3


In [16]:
model.fit(X=X_train, y=one_hot_y_train, max_iter=10000)

100%|██████████| 10000/10000 [00:14<00:00, 705.01it/s]


In [17]:
model.batch

120

In [19]:
model.batch=len(X_test)
loss, out = model.evaluate(X=X_test, truth=one_hot_y_test, verbose=True)

 30/30  |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.004
Prediction on 30 samples took 0.0 sec


In [20]:
truth = from_categorical(one_hot_y_test)
predicted = from_categorical(out)
performance=mean_accuracy_score(truth, predicted)

print('\nLoss Score: {:.3f}'.format(loss))
print('Accuracy Score: {:.3f}'.format(performance))


Loss Score: 0.004
Accuracy Score: 0.633


In [21]:
model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Connected_layer(outputs=num_classes, activation='logistic'))
model.add(Cost_layer(cost_type='mse'))

#model.compile(optimizer=SGD(lr=0.1, decay=0.03, lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])
model.summary()

layer       filters  size              input                output
   0 input                   120 x   1 x   1 x   4   ->   120 x   1 x   1 x   4
   1 connected               120 x   1 x   1 x   4   ->   120 x   3
   2 cost                    120 x   1 x   1 x   3   ->   120 x   1 x   1 x   3


In [22]:
model.fit(X=X_train, y=one_hot_y_train, max_iter=10000,verbose=False)

100%|██████████| 10000/10000 [00:08<00:00, 1221.78it/s]


In [23]:
truth = from_categorical(one_hot_y_test)
predicted = from_categorical(out)
performance=mean_accuracy_score(truth, predicted)

print('\nLoss Score: {:.3f}'.format(loss))
print('Accuracy Score: {:.3f}'.format(performance))


Loss Score: 0.004
Accuracy Score: 0.633


## Images

In [ ]:
images=image.load_images('data/digits')

In [ ]:
data=image.images_to_vectors(images)

In [ ]:
data.vectors/=255
summary(data)

In [ ]:
data_train,data_test=split(data)

In [ ]:
X_train=data_train.vectors
y_train=data_train.targets
one_hot_y_train=to_categorical(y_train)

X_test=data_test.vectors
y_test=data_test.targets
one_hot_y_test=to_categorical(y_test)

In [ ]:
one_hot_y_test[:10,:]

In [ ]:
# Reshape the data according to a 4D tensor
num_samples, size = X_test.shape
num_samples, size

num_classes=one_hot_y_test.shape[1]

X_test = X_test.reshape(num_samples, 1, 1, size)
one_hot_y_test = one_hot_y_test.reshape(num_samples,1,1,-1)

X_test.shape,one_hot_y_test.shape

In [ ]:
# Reshape the data according to a 4D tensor
num_samples, size = X_train.shape
num_samples, size

num_classes=one_hot_y_train.shape[1]

X_train = X_train.reshape(num_samples, 1, 1, size)
one_hot_y_train = one_hot_y_train.reshape(num_samples,1,1,-1)

X_train.shape,one_hot_y_train.shape

In [ ]:
batch=len(y_train)
step = batch

In [ ]:
model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Connected_layer(outputs=num_classes, activation='logistic'))
model.add(Cost_layer(cost_type='mse'))

#model.compile(optimizer=SGD(lr=0.1, decay=0.03, lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])
model.summary()

In [ ]:
model.fit(X=X_train, y=one_hot_y_train, max_iter=10000,verbose=False)

In [ ]:
model.batch=len(X_test)
loss, out = model.evaluate(X=X_test, truth=one_hot_y_test, verbose=True)

In [ ]:
truth = from_categorical(one_hot_y_test)
predicted = from_categorical(out)
performance=mean_accuracy_score(truth, predicted)

print('\nLoss Score: {:.3f}'.format(loss))
print('Accuracy Score: {:.3f}'.format(performance))

## convo net

In [ ]:
from NumPyNet.layers.convolutional_layer import Convolutional_layer
from NumPyNet.layers.maxpool_layer import Maxpool_layer
from NumPyNet.layers.softmax_layer import Softmax_layer
from NumPyNet.layers.batchnorm_layer import BatchNorm_layer


In [ ]:
batch=128

model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Convolutional_layer(size=3, filters=32, stride=1, pad=True, activation='Relu'))
model.add(BatchNorm_layer())
model.add(Maxpool_layer(size=2, stride=1, padding=True))
model.add(Connected_layer(outputs=100, activation='Relu'))
model.add(BatchNorm_layer())
model.add(Connected_layer(outputs=num_classes, activation='Linear'))
model.add(Softmax_layer(spatial=True, groups=1, temperature=1.))
model.add(Cost_layer(cost_type=cost_type.mse))

# model.compile(optimizer=SGD(lr=0.01, decay=0., lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])
model.summary()

In [ ]:
X_train.shape[1:]